In [ ]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, display
import matplotlib.image as img

plt.ioff()

In [ ]:
def is_energy(is_grid,J,h):
    e = -1 * J * (np.sum(np.multiply(is_grid,np.roll(is_grid,1,0)))+np.sum(np.sum(np.multiply(is_grid,np.roll(is_grid,1,1))))) - np.sum(h * is_grid)
    return e
def magnetization(is_grid):
    return np.abs(np.sum(is_grid)/(is_grid.shape[0]*is_grid.shape[1]))
def is_update(is_grid,J,h,temp):
    grid_candidate = is_grid.copy()
    x = np.random.randint(grid_candidate.shape[0])
    y = np.random.randint(grid_candidate.shape[1])
    grid_candidate[x][y] = -1* grid_candidate[x][y] 
    delta_e = is_energy(grid_candidate,J,h) - is_energy(is_grid,J,h)
    if delta_e < 0:     
        is_grid = grid_candidate
    elif np.random.rand() < np.exp(-1*delta_e/temp):
        is_grid = grid_candidate
    return is_grid
def updatefig(i):
    global is_grid
    is_grid = is_update(is_grid,J,h,temp)
    if i % 1000 == 0:
        pass
    plot.set_array(is_grid)
    return plot
def thermalize(is_grid,J,h,temp,N):
    for n in range(N):
        is_grid = is_update(is_grid,J,h,temp)
        if n % 1000 == 0:
            pass
    return is_grid
def measure(is_grid,J,h,temp,N,Td):
    e = []
    e2 = []
    M = []
    M2 =[]
    for n in range(N):
        is_grid = is_update(is_grid,J,h,temp)
        if n % Td == 0:
            e.append(is_energy(is_grid,J,h))
            e2.append(np.power(is_energy(is_grid,J,h),2))
            M.append(magnetization(is_grid))
            M2.append(np.power(magnetization(is_grid),2))
        energy = np.average(e)
        avg_magnetization = np.average(M)
        capacity = np.average(e2)-np.power(np.average(e),2)
        susceptibility = np.average(M2)-np.power(np.average(M),2)
    return energy,avg_magnetization,susceptibility,capacity

In [ ]:
L = 30
is_grid = np.random.choice([-1,1],(L,L))

temp = 0.5
h = 0
J = 1

fig = plt.figure()
plot = plt.imshow(is_grid,cmap='bwr',interpolation='none')
anim = animation.FuncAnimation(fig,updatefig, frames = 10000, interval = 1, repeat=False)
plt.show()

In [ ]:
e = []
M = []
H = []
Cp = []
h = 0
J = 1
L = 25
temps = np.linspace(0.3,3.55,17)
for temp in temps:
    print(temp)
    is_grid = np.random.choice([-1,1],(L,L))
    is_grid = thermalize(is_grid,J,h,temp,1000000)
    ev,Mv,Hv,Cpv = measure(is_grid,J,h,temp,1000000,1000)
    e.append(ev)
    M.append(Mv)
    H.append(Hv)
    Cp.append(Cpv)

In [ ]:
fig = plt.figure()
fig.suptitle('Właściwości statystyczne układu w funkcji temperatury')
ax1 =plt.subplot2grid((2,2),(0,0))
ax1.set_title('Energia')
ax1.set_ylabel('Energia')
ax1.set_xlabel('Temperatura')
plot1 = plt.plot(temps,e)
ax2 = plt.subplot2grid((2,2),(0,1))
ax2.set_title('Magnetyzacja')
ax2.set_ylabel('Magnetyzacja')
ax2.set_xlabel('Temperatura')
plot2 = plt.plot(temps,M)
ax3 = plt.subplot2grid((2,2),(1,0))
ax3.set_title('Podatność magnetyczna')
ax3.set_ylabel('Podatność magnetyczna')
ax3.set_xlabel('Temperatura')
plot3 = plt.plot(temps,H)
ax4 = plt.subplot2grid((2,2),(1,1))
plot4 = plt.plot(temps,Cp)
ax4.set_title('Ciepło właściwe')
ax4.set_ylabel('Ciepło właściwe')
ax4.set_xlabel('Temperatura')
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

In [ ]:
fig.savefig('ising.png',dpi=800)

In [ ]:
im = img.imread('ising.png')
plt.figure()
plt.imshow(im)
plt.show()